In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# 전처리
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

# 모델
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# 평가
from sklearn.metrics import accuracy_score, f1_score, classification_report

import warnings
warnings.filterwarnings('ignore')

## 2. 데이터 로드 & 시간순 정렬

**핵심**: 시계열 데이터이므로 반드시 시간순 정렬

In [ ]:
# 데이터 로드
DATA_PATH = "new_data/dataset_200k_ready.csv"
df = pd.read_csv(DATA_PATH)

# DateTime 파싱 & 시간순 정렬 (중요)
df['DateTime'] = pd.to_datetime(df['DateTime'])
df = df.sort_values('DateTime').reset_index(drop=True)

print(f"데이터: {len(df):,}건")
print(f"유저 수: {df['User'].nunique()}명")
print(f"카테고리: {df['NextCategory'].unique()}")

## 3. Train/Test 분할 (시간순)

⚠️ **중요**: 랜덤 분할 ❌, 시간순 분할 ✅

- 과거 80% → Train (학습)
- 미래 20% → Test (검증)

**이유**: 미래 데이터로 과거 예측하면 데이터 누출

In [ ]:
# 시간순 분할 (80:20)
split_idx = int(len(df) * 0.8)

train_df = df.iloc[:split_idx].copy()   # 과거 80%
test_df = df.iloc[split_idx:].copy()    # 미래 20%

print(f"Train: {len(train_df):,}건 (과거 데이터)")
print(f"Test:  {len(test_df):,}건 (미래 데이터)")

## 4. 피처 엔지니어링

### 4-1. 시간/금액 기반 피처

현재 거래 시점의 정보만 사용 → 데이터 누출 없음

In [ ]:
def add_time_features(df):
    """
    시간/금액 기반 피처 추가
    - 현재 거래 시점 정보만 사용 (데이터 누출 없음)
    """
    df = df.copy()
    
    # 주말 여부
    df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
    
    # 시간대별 플래그
    df['IsLunchTime'] = ((df['Hour'] >= 11) & (df['Hour'] <= 14)).astype(int)    # 점심 (11-14시)
    df['IsEvening'] = ((df['Hour'] >= 18) & (df['Hour'] <= 21)).astype(int)      # 저녁 (18-21시)
    df['IsMorningRush'] = ((df['Hour'] >= 7) & (df['Hour'] <= 9)).astype(int)    # 출근 (7-9시)
    
    # 금액 구간 (저가/중가/고가/초고가)
    df['AmountBin'] = pd.cut(
        df['Amount_Clean'], 
        bins=[-float('inf'), 20, 50, 100, float('inf')],
        labels=['저가', '중가', '고가', '초고가']
    )
    
    return df

# Train/Test 각각 적용
train_df = add_time_features(train_df)
test_df = add_time_features(test_df)

print("✅ 시간/금액 피처 추가 완료")
print(f"   - IsWeekend, IsLunchTime, IsEvening, IsMorningRush, AmountBin")

### 4-2. 사용자 통계 피처 (⚠️ 데이터 누출 방지)

**핵심**: Train에서만 통계 계산 → Test에 적용

❌ 잘못된 방법: 전체 데이터에서 통계 계산 (미래 정보 포함)

✅ 올바른 방법: Train에서만 계산 → Test에 merge

In [ ]:
# ⚠️ Train 데이터에서만 사용자 통계 계산

# 1. 사용자별 기본 통계 (Train에서만)
user_stats = train_df.groupby('User').agg({
    'Amount_Clean': ['mean', 'std', 'count'],
    'Category': lambda x: x.value_counts().index[0]  # 최빈 카테고리
}).reset_index()

user_stats.columns = ['User', 'User_AvgAmount', 'User_StdAmount', 'User_TxCount', 'User_FavCategory']

print("사용자 통계 (Train 기준):")
print(user_stats.head(3))

In [ ]:
# 2. 사용자별 카테고리 이용 비율 (Train에서만)
cat_freq = train_df.groupby(['User', 'Category']).size().unstack(fill_value=0)
cat_freq = cat_freq.div(cat_freq.sum(axis=1), axis=0)  # 비율로 변환
cat_freq.columns = [f'User_{c}_Ratio' for c in cat_freq.columns]
cat_freq = cat_freq.reset_index()

print("카테고리별 이용 비율:")
print(cat_freq.head(3))

In [ ]:
# 3. Train/Test에 통계 적용
#    - Train: Train 통계 적용
#    - Test:  Train 통계 적용 (미래 정보 사용 ❌)

train_df = train_df.merge(user_stats, on='User', how='left')
train_df = train_df.merge(cat_freq, on='User', how='left')

test_df = test_df.merge(user_stats, on='User', how='left')  # Train 통계를 Test에 적용
test_df = test_df.merge(cat_freq, on='User', how='left')

# 4. 누락값 처리 (Test에 새 사용자가 있을 경우)
for col in test_df.columns:
    if col.startswith('User_') and col.endswith('_Ratio'):
        test_df[col] = test_df[col].fillna(0)
    elif col in ['User_AvgAmount', 'User_StdAmount', 'User_TxCount']:
        test_df[col] = test_df[col].fillna(test_df[col].median())

print("✅ 사용자 통계 피처 적용 완료")
print("   ⚠️ Train에서만 계산 → Test에 적용 (데이터 누출 방지)")

## 5. 피처 정의 (총 21개)

In [ ]:
# 피처 정의 (총 21개)

# 기존 피처 (6개)
base_numeric = ['Hour', 'DayOfWeek', 'Amount_Clean', 'TimeSincePrev']
base_categorical = ['Category', 'TimeSlot']

# 시간/금액 피처 (5개) - 새로 추가
time_numeric = ['IsWeekend', 'IsLunchTime', 'IsEvening', 'IsMorningRush']
time_categorical = ['AmountBin']

# 사용자 통계 피처 (10개) - 새로 추가
user_numeric = ['User_AvgAmount', 'User_StdAmount', 'User_TxCount']
user_numeric += [c for c in train_df.columns if c.startswith('User_') and c.endswith('_Ratio')]
user_categorical = ['User_FavCategory']

# 전체 피처
all_numeric = base_numeric + time_numeric + user_numeric
all_categorical = base_categorical + time_categorical + user_categorical

print(f"📊 피처 구성:")
print(f"   - 기존: {len(base_numeric + base_categorical)}개")
print(f"   - 시간/금액: +{len(time_numeric + time_categorical)}개")
print(f"   - 사용자 통계: +{len(user_numeric + user_categorical)}개")
print(f"   - 총: {len(all_numeric + all_categorical)}개")

## 6. 데이터 준비

In [ ]:
# X (피처)
X_train = train_df[all_numeric + all_categorical]
X_test = test_df[all_numeric + all_categorical]

# y (타겟) - 레이블 인코딩
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['NextCategory'])
y_test = label_encoder.transform(test_df['NextCategory'])

print(f"X_train: {X_train.shape}")
print(f"X_test:  {X_test.shape}")
print(f"클래스: {label_encoder.classes_}")

## 7. 전처리 파이프라인

- 수치형: 결측치 → 중앙값, 스케일링
- 범주형: 결측치 → 최빈값, 원핫인코딩

In [ ]:
def build_pipeline(model):
    """
    전처리 + 모델 파이프라인 생성
    """
    transformers = [
        # 수치형 피처: 결측치(중앙값) → 스케일링
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')), 
            ('scaler', StandardScaler())
        ]), all_numeric),
        
        # 범주형 피처: 결측치(최빈값) → 원핫인코딩
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')), 
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), all_categorical)
    ]
    
    return Pipeline([
        ('preprocess', ColumnTransformer(transformers)), 
        ('model', model)
    ])

print("✅ 파이프라인 함수 정의 완료")

## 8. 모델 정의 (8개)

**핵심 설정**: `class_weight='balanced'`
- 불균형 데이터 자동 보정
- 소수 클래스(쇼핑 5.7%)에 높은 가중치 부여

In [ ]:
# 8개 모델 정의
models = {
    # LightGBM 계열
    'LightGBM': LGBMClassifier(
        n_estimators=300, 
        learning_rate=0.05, 
        max_depth=6, 
        class_weight='balanced',  # 불균형 보정
        random_state=42, 
        verbose=-1
    ),
    
    'LightGBM_XL': LGBMClassifier(
        n_estimators=600, 
        learning_rate=0.015, 
        max_depth=15, 
        num_leaves=128,
        class_weight='balanced',
        random_state=42, 
        verbose=-1
    ),
    
    # XGBoost 계열
    'XGBoost': XGBClassifier(
        n_estimators=300, 
        learning_rate=0.05, 
        max_depth=6, 
        random_state=42, 
        verbosity=0
    ),
    
    'XGBoost_Deep': XGBClassifier(
        n_estimators=400, 
        learning_rate=0.03, 
        max_depth=10, 
        random_state=42, 
        verbosity=0
    ),
    
    # RandomForest 계열
    'RandomForest': RandomForestClassifier(
        n_estimators=200, 
        max_depth=15, 
        class_weight='balanced',
        random_state=42
    ),
    
    'RandomForest_Deep': RandomForestClassifier(
        n_estimators=300, 
        max_depth=25, 
        class_weight='balanced_subsample',
        random_state=42
    ),
    
    # HistGradientBoosting (빠름)
    'HistGB': HistGradientBoostingClassifier(
        max_iter=300, 
        learning_rate=0.05, 
        max_depth=10, 
        class_weight='balanced',
        random_state=42
    ),
    
    # ExtraTrees (🏆 최고 성능)
    'ExtraTrees': ExtraTreesClassifier(
        n_estimators=200, 
        max_depth=15, 
        class_weight='balanced',
        random_state=42
    ),
}

print(f"✅ {len(models)}개 모델 정의 완료")
for name in models:
    print(f"   - {name}")

## 9. 모델 학습 & 평가

In [ ]:
import time

results = []

print("=" * 70)
print(f"{'모델':>20} | {'Accuracy':>10} | {'Macro F1':>10} | {'시간':>8}")
print("=" * 70)

for name, model in models.items():
    start = time.time()
    
    # 파이프라인 생성 & 학습
    pipeline = build_pipeline(model)
    pipeline.fit(X_train, y_train)
    
    # 예측
    y_pred = pipeline.predict(X_test)
    
    # 평가
    acc = accuracy_score(y_test, y_pred) * 100
    macro_f1 = f1_score(y_test, y_pred, average='macro') * 100
    elapsed = time.time() - start
    
    results.append({
        'Model': name, 
        'Accuracy': acc, 
        'Macro_F1': macro_f1, 
        'Time': elapsed
    })
    
    print(f"{name:>20} | {acc:>9.2f}% | {macro_f1:>9.2f}% | {elapsed:>7.2f}s")

print("=" * 70)

## 10. 최종 결과 (Macro F1 기준 정렬)

In [ ]:
# 결과 정렬
results_df = pd.DataFrame(results).sort_values('Macro_F1', ascending=False)

print("\n" + "=" * 70)
print("🏆 최종 순위 (Macro F1 기준)")
print("=" * 70)

for i, (_, row) in enumerate(results_df.iterrows(), 1):
    marker = " ← BEST!" if i == 1 else ""
    print(f"{i}. {row['Model']:20s} | Accuracy: {row['Accuracy']:.2f}% | Macro F1: {row['Macro_F1']:.2f}%{marker}")

# 표로 출력
results_df

## 11. 최고 모델 상세 분석 (ExtraTrees)

In [ ]:
# 최고 모델로 다시 학습
best_model = ExtraTreesClassifier(
    n_estimators=200, 
    max_depth=15, 
    class_weight='balanced',
    random_state=42
)

pipeline = build_pipeline(best_model)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# 카테고리별 성능
print("📊 카테고리별 성능 (ExtraTrees)")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

## 12. 피처 중요도 (Feature Importance)

In [ ]:
# 피처 이름 추출
preprocessor = pipeline.named_steps['preprocess']
feature_names = []

for name, transformer, columns in preprocessor.transformers_:
    if name == 'num':
        feature_names.extend(columns)
    elif name == 'cat':
        encoder = transformer.named_steps['encoder']
        cat_features = encoder.get_feature_names_out(columns)
        feature_names.extend(cat_features)

# 중요도 추출
importances = pipeline.named_steps['model'].feature_importances_

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

# Top 15 출력
print("📊 피처 중요도 Top 15")
print("=" * 50)
for i, (_, row) in enumerate(importance_df.head(15).iterrows(), 1):
    print(f"{i:2d}. {row['Feature']:30s}: {row['Importance']*100:.2f}%")

In [ ]:
# 시각화 (matplotlib)
import matplotlib.pyplot as plt

top15 = importance_df.head(15).sort_values('Importance', ascending=True)

plt.figure(figsize=(10, 8))
plt.barh(top15['Feature'], top15['Importance'] * 100, color='steelblue')
plt.xlabel('Importance (%)')
plt.title('Feature Importance Top 15 (ExtraTrees)')
plt.tight_layout()
plt.show()

---

## 📌 핵심 요약

### 1. 타겟 생성 (다음 라벨 예측)
```python
df['NextCategory'] = df.groupby('User')['Category'].shift(-1)
```
- `shift(-1)`: 다음 행의 카테고리 → 미래 예측!

### 2. 데이터 누출 방지
- **시간순 분할**: 과거 80% → Train, 미래 20% → Test
- **사용자 통계**: Train에서만 계산 → Test에 적용

### 3. 피처 엔지니어링 (+5.33%p 향상)
- 기존 6개 → 21개 피처
- 사용자 통계 피처가 효과적!

### 4. 최종 성과
| 항목 | 값 |
|------|-----|
| 모델 | ExtraTrees |
| Macro F1 | **54.86%** |
| Accuracy | **63.09%** |
| 피처 수 | 21개 |